Spousteni pyspark

`pyspark --num-executors 2 --executor-memory 1500M --packages com.databricks:spark-csv_2.10:1.5.0 --master yarn`

In [ ]:
# uzitecne importy

from pyspark.sql.types import *
from pyspark.sql import functions as F
import re
from itertools import cycle

1 Načtěte soubor se stopwords.

2 Načtěte písňové texty z CSV přímo do DataFrame s definicí schématu, nakešujte:

cesta `/user/pascepet/lyrics_data/`

oddelovac `','`

schema

```
id – long
album – string
rok – integer
interpret – string
zanr – string
text – string
```

3 Přidejte pole `poc_slov`, které bude obsahovat počet slov v textu příslušné písně (slovo je část řetězce mezi dvěma mezerami). 
Obtížnější varianta: přidejte pole `pocet_slov_unik`, které bude obsahovat počet unikátních slov po převedení na malá písmena a vyhození znaků `,.:;!?()[]`


4 Spočtěte za každého interpreta průměrný počet slov v textu a vypište 20 interpretů s nejvyšším průměrem, ale jen takových, kteří mají v datech aspoň 10 písní.


5 Zjistěte, kteří interpreti mají v datech aspoň 500 písní.

6 Najděte 20 nejčastěji se vyskytujících slov, která nejsou v stopwords. Každé slovo se počítá tolikrát, kolikrát v textu je. (Obtížnější varianta: každé slovo se počítá nejvýše jednou za každou píseň.)

7 Vybrat z nejčastěji se vyskytujících slov pět charakteristických a k DataFrame s písněmi přidejte pro každé vybrané slovo příznak, zda se dané slovo v dané písni vyskytuje (0/1 nebo False/True).

8 Vyberte z interpretů s nejvyššími zastoupením pět a pro každého spočtěte podíl výskytu každého z pěti slov vybraných výše (podíl = počet písní s výskytem slova / počet všech písní interpreta). Najděte dvojice interpretů s nejpodobnějšími a nejméně podobnými podíly.